In [1]:
#https://towardsdatascience.com/logistic-regression-using-python-sklearn-numpy-mnist-handwriting-recognition-matplotlib-a6b31e2b166a
import pandas as pd
import unicodecsv
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn import preprocessing

plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import seaborn as sns
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

/Users/rahul/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

def read_csv(data):
#""" Returns a list of dicts from .csv file passed in as data"""
    with open(data, "rb") as f:
        reader = list(unicodecsv.DictReader(f))
        return reader

file = read_csv('/Users/rahul/Desktop/TalkingData AdTracking Fraud Detection Challenge/Data/mnt/ssd/kaggle-talkingdata2/competition_files/train_sample.csv') # call the function and pass in your 'filename'
#use option+command+c to copy path name
train_small= pd.DataFrame(file) # call pd.DataFrame() function with file as an argument 
                   # to convert it to DataFrame object
test_data=pd.DataFrame(read_csv('/Users/rahul/Desktop/TalkingData AdTracking Fraud Detection Challenge/Data/test.csv'))

In [3]:
train_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
ip                 100000 non-null object
app                100000 non-null object
device             100000 non-null object
os                 100000 non-null object
channel            100000 non-null object
click_time         100000 non-null object
attributed_time    100000 non-null object
is_attributed      100000 non-null object
dtypes: object(8)
memory usage: 6.1+ MB


In [4]:
train_small.describe()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
count,100000,100000,100000,100000,100000,100000,100000,100000
unique,34857,161,100,130,161,80350,228,2
top,5348,3,1,19,280,2017-11-08 12:01:02,,0
freq,669,18279,94338,23870,8114,7,99773,99773


In [5]:
test_data.describe()

,click_id,ip,app,device,os,channel,click_time
count,18790469,18790469,18790469,18790469,18790469,18790469,18790469
unique,18790469,93936,417,1985,395,178,21603
top,9097443,5348,9,1,19,107,2017-11-10 09:00:10
freq,1,182522,2872176,17360269,4334532,1214650,1466


In [6]:
train_small.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,,0
1,105560,25,1,17,259,2017-11-07 13:40:27,,0
2,101424,12,1,19,212,2017-11-07 18:05:24,,0
3,94584,13,1,13,477,2017-11-07 04:58:08,,0
4,68413,12,1,1,178,2017-11-09 09:00:09,,0


In [7]:
train_small.dtypes #display different types of object

ip                 object
app                object
device             object
os                 object
channel            object
click_time         object
attributed_time    object
is_attributed      object
dtype: object

In [8]:
train_small['click_time'] =  pd.to_datetime(train_small['click_time'], format='%Y-%m-%d %H:%M:%S')
#converted the object format to datetime64 format for column click_time
#train_small.dtypes #use this to check if the change happened or not

#now do the same for attributed_time
train_small['attributed_time'] =  pd.to_datetime(train_small['attributed_time'], format='%Y-%m-%d %H:%M:%S')
train_small['device']  = train_small['device'].astype('category')
train_small['app']  = train_small['app'].astype('category')
train_small['os']  = train_small['os'].astype('category')
train_small['channel']  = train_small['channel'].astype('category')
train_small['ip']  = train_small['ip'].astype('category')
train_small['Difference'] = train_small['attributed_time'].sub(train_small['click_time'], axis=0) #diff of dates

#train_small.sort_values(by=['attributed_time'], ascending=[False])
#to plot histogram, but since the number of categories are high the graph looks redundant
#train_small['device'].value_counts().plot(kind='bar')
#plt.show()

#to extract specific day of month from datetime object
#train_small['click_time_month']  = pd.DatetimeIndex(train_small['click_time'] ).month


train_small.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,Difference
0,87540,12,1,13,497,2017-11-07 09:30:38,NaT,0,NaT
1,105560,25,1,17,259,2017-11-07 13:40:27,NaT,0,NaT
2,101424,12,1,19,212,2017-11-07 18:05:24,NaT,0,NaT
3,94584,13,1,13,477,2017-11-07 04:58:08,NaT,0,NaT
4,68413,12,1,1,178,2017-11-09 09:00:09,NaT,0,NaT


In [9]:
test_data.head() #notice that the test_data does not have 'attributed_time' column; 
#this means we need to remove 'attributed_time' &'difference' column from train data

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [10]:
del train_small['attributed_time'] #deleting columns from data frame
del train_small['Difference']

In [11]:
test_data['click_time'] =  pd.to_datetime(test_data['click_time'], format='%Y-%m-%d %H:%M:%S')
#converted the object format to datetime64 format for column click_time
#train_small.dtypes #use this to check if the change happened or not

#now do the same for attributed_time
test_data['device']  = test_data['device'].astype('category')
test_data['app']  = test_data['app'].astype('category')
test_data['os']  = test_data['os'].astype('category')
test_data['channel']  = test_data['channel'].astype('category')
test_data['ip']  = test_data['ip'].astype('category')
test_data.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [12]:
train_small.head()

,ip,app,device,os,channel,click_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,0
1,105560,25,1,17,259,2017-11-07 13:40:27,0
2,101424,12,1,19,212,2017-11-07 18:05:24,0
3,94584,13,1,13,477,2017-11-07 04:58:08,0
4,68413,12,1,1,178,2017-11-09 09:00:09,0


In [13]:
#subsetting the test data to remove click_id
test_data_1 = test_data[['ip', 'app', 'device', 'os', 'channel', 'click_time']]

In [14]:
#cols=list(train_small.columns.values)
cols= ['ip', 'app', 'device', 'os', 'channel', 'click_time']
target = 'is_attributed'

In [15]:
print(cols)

['ip', 'app', 'device', 'os', 'channel', 'click_time']


In [16]:
train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time']

In [17]:
train_small['click_time_day']  = pd.DatetimeIndex(train_small['click_time'] ).day #new day feature

In [18]:
test_data_1['click_time_day']  = pd.DatetimeIndex(test_data_1['click_time']).day

In [19]:
train_small['click_time_hour']  = pd.DatetimeIndex(train_small['click_time'] ).hour #new hour feature
test_data_1['click_time_hour']  = pd.DatetimeIndex(test_data_1['click_time']).hour

In [20]:
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression()

In [23]:
x_train=train_small[['ip', 'app', 'device', 'os', 'channel','click_time_day','click_time_hour']] #removing cick_time as it is in timestamp
y_train=train_small[['is_attributed']]

In [24]:
logisticRegr.fit(x_train, y_train) #training the model

/Users/rahul/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
test_data_1=test_data_1[['ip', 'app', 'device', 'os', 'channel','click_time_day','click_time_hour']]

In [29]:
predictions = logisticRegr.predict_proba(test_data_1)

In [32]:
prob_one = [i[1] for i in predictions]

In [33]:
prob_zero = [i[0] for i in predictions]

In [36]:
submission = pd.DataFrame({'click_id': test_data.click_id, 'is_attributed': prob_one})

In [37]:
submission.head()

,click_id,is_attributed
0,0,0.000489
1,1,0.000130
2,2,0.001014
3,3,0.001097
4,4,0.000370


In [39]:
submission.to_csv('submission.csv', index = False)

In [40]:
submission_with_zero = pd.DataFrame({'click_id': test_data.click_id, 'is_attributed': prob_zero})

In [41]:
submission_with_zero.to_csv('submission_with_zero.csv', index = False)